In [ ]:
import geemap
import ee
from Monarch import user_gee
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import savgol_filter
from matplotlib import rcParams

In [ ]:
Map = geemap.Map()
Map

In [ ]:
roi = geemap.geojson_to_ee(r'H:/Monarch/paper/Data/area.geojson').geometry()
Map.center_object(roi, 12)
Map.addLayer(roi, {}, 'roi')

In [ ]:
visParams = {'max': 3000, 'min': 0, 'bands': ['B4', 'B3', 'B2']}

In [ ]:
for year in range(2018, 2019):
    s2_year = []
    for i in range(3, 11):
        s2 = ee.ImageCollection("COPERNICUS/S2")\
                .filterBounds(roi)\
                .sort('CLOUD_COVERAGE_ASSESSMENT', False)\
                .filterDate(str(year), str(year+1))\
                .filter(ee.Filter.calendarRange(i, i, 'month'))\
                .map(user_gee.rm_cloud_s2_sr)\
                .mosaic().clip(roi)
        Map.addLayer(s2, visParams, f"s2_{i}")
        s2_year.append(s2)
    s2_year = ee.ImageCollection(s2_year)

In [ ]:
year = 2018
modis = ee.ImageCollection("MODIS/006/MOD09A1").filterDate(str(year), str(year+1))\
            .map(lambda img: img.clip(roi))
ndvi = modis.map(lambda img: img.normalizedDifference(['sur_refl_b02', 'sur_refl_b01'])\
                 .rename(['ndvi']).set({'system:id':img.get('system:id'), 
                                        'system:time_start':img.get('system:time_start')}))
lswi = modis.map(lambda img: img.normalizedDifference(['sur_refl_b02', 'sur_refl_b06'])\
                 .rename(['lswi']).set({'system:id':img.get('system:id'), 
                                        'system:time_start':img.get('system:time_start')}))

In [ ]:
year = 2018
modis = ee.ImageCollection("MODIS/006/MOD09A1").filterDate(str(year), str(year+1))\
            .map(lambda img: img.clip(roi))
ndvi = modis.map(lambda img: img.normalizedDifference(['sur_refl_b02', 'sur_refl_b01'])\
                 .rename(['ndvi']))
lswi = modis.map(lambda img: img.normalizedDifference(['sur_refl_b02', 'sur_refl_b06'])\
                 .rename(['lswi']).set({'system:id':img.get('system:id'), 
                                        'system:time_start':img.get('system:time_start')}))

In [ ]:
ndvi_sg_list = user_gee.sg_images(ndvi, 11, 3)
lswi_sg_list = user_gee.sg_images(lswi, 11, 3)

In [ ]:
rice_points = geemap.geojson_to_ee(r'H:/Monarch/paper/Data/rice_ps.geojson')

In [ ]:
ndvi_img = ndvi.toBands().rename([f'ndvi_{i}' for i in range(46)])
ndvi_training = ndvi_img.sampleRegions(**{
  'collection': rice_points,
  'scale': 500
})
lswi_img = lswi.toBands().rename([f'lswi_{i}' for i in range(46)])
lswi_training = lswi_img.sampleRegions(**{
  'collection': rice_points,
  'scale': 500
})

In [ ]:
ndvi_data = [ndvi_training.aggregate_array(f'ndvi_{i}').getInfo() for i in range(46)]
lswi_data = [lswi_training.aggregate_array(f'lswi_{i}').getInfo() for i in range(46)]

In [ ]:
ndvi_data = np.array(ndvi_data)
lswi_data = np.array(lswi_data)
ndvi_sg_data = savgol_filter(ndvi_data, 11, 3, axis=0)
lswi_sg_data = savgol_filter(lswi_data, 11, 3, axis=0)

In [ ]:
ndvi_sg_data.mean(1)

In [ ]:
config = {
    "font.family": 'serif',
    "font.size": 14,
    "mathtext.fontset": 'stix',
    "font.serif": 'Simsun',
}
rcParams.update(config)
fig, ax = plt.subplots(figsize=(12, 8))
ax.set_xlabel('时相')
ax.set_ylabel('NDVI值')
ax.plot([i for i in range(46)], ndvi_data.mean(1), '-o', color='blue', lw=1, label='原始的NDVI值')
ax.plot([i for i in range(46)], ndvi_sg_data.mean(1), '-^', color='r', lw=1, label='sg滤波后的NDVI值')
ax.legend(loc=2, frameon=False, handlelength=2)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
ax.set_xlabel('时相')
ax.set_ylabel('LSWI值')
ax.plot([i for i in range(46)], lswi_data.mean(1), '-o', color='blue', lw=1, label='原始的LSWI值')
ax.plot([i for i in range(46)], lswi_sg_data.mean(1), '-^', color='r', lw=1, label='sg滤波后的LSWI值')
ax.legend(loc=0, frameon=False, handlelength=2, )

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
ax.set_xlabel('时相')
ax.set_ylabel('LSWI值')
ax.plot([i for i in range(46)], lswi_means, '-o', color='blue', lw=1, label='原始的LSWI值')
ax.plot([i for i in range(46)], savgol_filter(lswi_means, 7, 3), '-^', color='r', lw=1, label='sg滤波后的LSWI值')
ax.legend(loc=2, frameon=False, handlelength=2)